<a href="https://colab.research.google.com/github/iued-uni-heidelberg/projTermiKnowledge/blob/main/TermiKnowledgeDemo03ExtractingTerms.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# This notebook show some methods of terminology extraction

1. Installing Part-of-Speech tagger for languages we need:

English , German, Italian, Czech, Polish, 

In [ ]:
%%bash
mkdir treetagger
cd treetagger
# Download the tagger package for your system (PC-Linux, Mac OS-X, ARM64, ARMHF, ARM-Android, PPC64le-Linux).
wget https://cis.lmu.de/~schmid/tools/TreeTagger/data/tree-tagger-linux-3.2.4.tar.gz
tar -xzvf tree-tagger-linux-3.2.4.tar.gz
# Download the tagging scripts into the same directory.
wget https://cis.lmu.de/~schmid/tools/TreeTagger/data/tagger-scripts.tar.gz
gunzip tagger-scripts.tar.gz
# Download the installation script install-tagger.sh.
wget https://cis.lmu.de/~schmid/tools/TreeTagger/data/install-tagger.sh
# Download the parameter files for the languages you want to process.
# list of all files (parameter files) https://cis.lmu.de/~schmid/tools/TreeTagger/#parfiles
wget https://cis.lmu.de/~schmid/tools/TreeTagger/data/english.par.gz
sh install-tagger.sh
cd ..
sudo pip install treetaggerwrapper

In [ ]:
%%bash
wget https://heibox.uni-heidelberg.de/f/95a3875771c040db959a/?dl=1
mv index.html?dl=1 humanrights02.txt

wget https://heibox.uni-heidelberg.de/f/a79b829e15c24dbd9e77/?dl=1
mv index.html?dl=1 covid3m-en.txt

wget https://heibox.uni-heidelberg.de/f/e3c1edbcec9649f5b4c4/?dl=1
mv index.html?dl=1 TED2020-enka-en.txt

In [ ]:
!./treetagger/cmd/tree-tagger-english humanrights02.txt >humanrights02_vert.txt

	reading parameters ...
	tagging ...
1545000	 finished.


In [ ]:
!head --lines=20 humanrights02_vert.txt

<doc id="file22593149" filename="108544.htm" parent_folder="web1" url="https://2009-2017.state.gov/j/drl/rls/irf/2008/108544.htm">
<p>
ArchivedContent	NP	<unknown>
</p>
<p>
Information	NP	Information
released	VBD	release
online	JJ	online
from	IN	from
January	NP	January
20	CD	@card@
,	,	,
2009	CD	@card@
to	TO	to
January	NP	January
20	CD	@card@
,	,	,
2017	CD	@card@
.	SENT	.
Note	NN	note
